## Validation of vaccine composition
### Imports and constants

In [ ]:
import pandas as pd
import numpy as np
import os

basepath = "/mnt/container-nle-neoagdt/data/"
suppdir = "tran2015_supp"
basedir = "tran2015"
vacfile = "analysis/vaccines/selected-vaccine-elements.budget-10.minsum.adaptive.csv"
predfile = "MHC_Class_I/{}.all_epitopes.tsv"

# Generate gene mapping dict
genemap = dict()
with open(os.path.join(basepath, 'genenames_mapping.txt'), 'r') as fh:
    for row in fh:
        columns = row.split('\t')
        genemap[columns[1].upper()] = columns[10] # alternatively column 12
        genemap[columns[10].upper()] = columns[10]
        for i in [4, 5, 7]:
            if columns[i] is not None:
                csplit = columns[i].split(',')
                for altname in csplit:
                        genemap[altname.strip().upper()] = columns[10]
samples = [x for x in os.listdir(os.path.join(basepath, basedir)) if x.isnumeric()]
# Excluding sample 3971 since it is not present in the supplementary mutation tables
samples.remove('3971')
# Excluding sample 4069 since it has too few mutations
samples.remove('4069')

### Functions

In [ ]:
def load_confirmed_mutations():
    return pd.read_csv(os.path.join(basepath, suppdir, 'tran_patients_tcellreaction.txt'), sep='\t')
def get_mutation_id(dfrow):
    return ''.join([dfrow['gene'], '_', dfrow['mutation'][0], '>', dfrow['mutation'][-1]])
def load_vaccine(sample):
    return pd.read_csv(os.path.join(basepath, basedir, sample, vacfile))
def get_mutations_pub(sample):
    return pd.read_excel(os.path.join(basepath, suppdir, 'tran_patients_mutations.xlsx'),
                       sheet_name=sample, skiprows=3, header=None)    
def get_mutations_sim(sample):
    return pd.read_csv(os.path.join(basepath, basedir, sample, 'variants.csv'))
def get_mutations_umap(sample):
    try:
        df = pd.read_csv(os.path.join(basepath, basedir, sample, 'unmapped_genes.txt'))
    except pd.errors.EmptyDataError:
        df = pd.DataFrame()
    return df
def load_selected_elements(sample):
    return pd.read_csv(os.path.join(basepath, basedir, sample, vacfile))
def load_predictions(sample):
    df = pd.read_csv(os.path.join(basepath, basedir, sample, predfile.format(sample)), sep='\t')
                       #usecols=['Mutation', 'Epitope Seq', 'NetMHC Score', 'NetMHC Percentile'])
    # Sample specific cleanup
    if sample == '3812':
        df['Mutation'] = df['Mutation'].apply(lambda x: x.replace('.', ''))
    return df

In [ ]:
def get_performance_scores(sim, t_cell_type="CD8"):
    sample_list = list()
    tp_list = list()
    fn_list = list()
    recall_list = list()
    results = list()
    
    for sample in samples:  # iterate patients
        vac_df = load_selected_elements(sample)
        vac_df = vac_df[vac_df["simulation_name"] == sim]

        gt_df = df_tcell[(df_tcell['sample'] == sample) & (df_tcell['type'] == t_cell_type)]
        gt_mutations = set(gt_df["Mutation ID"].unique())

        for i in range(10):  # iterate over repeated simulated cell populations
            sample_list.append(sample)

            vac_rep_df = vac_df[vac_df["repetition"] == i]
            predicted_mutations = set(vac_rep_df["peptide"].unique())

            if len(gt_mutations) == 0:
                tp_list.append(np.nan)
                fn_list.append(np.nan)
                recall_list.append(np.nan)
            else:
                tp = len(gt_mutations.intersection(predicted_mutations))
                fn = len(gt_mutations) - len(gt_mutations.intersection(predicted_mutations))
                recall = tp / (tp + fn)

                tp_list.append(tp)
                fn_list.append(fn)
                recall_list.append(recall)

            results.append(pd.DataFrame(data={
                "sample": sample_list,
                "tp": tp_list,
                "fn": fn_list,
                "recall": recall_list
            }))
    
    return {
        "mean": pd.concat(results).groupby('sample').mean(), 
        "std": pd.concat(results).groupby('sample').std()
    }

### Validation
#### T cell responses measured by Tran et al.

In [ ]:
df_tcell = load_confirmed_mutations()
df_tcell['gene'] = df_tcell['gene'].apply(
        lambda x: str(genemap[x.split('_')[0].upper()]))
df_tcell['Mutation ID'] = df_tcell.apply(get_mutation_id, axis=1)
df_tcell['sample'] = df_tcell['sample'].apply(lambda x: str(x))
# Remove reaction with unmappable gene
df_tcell.drop(index=16, inplace=True)
df_tcell

#### Genes with mutation that could not be mapped and were excluded from analysis

In [ ]:
sample_list = list()
pubmut_list = list()
unmap_list  = list()

for sample in samples:
    sample_list.append(sample)
    df_mp = get_mutations_pub(sample)
    df_um = get_mutations_umap(sample)
    pubmut_list.append(len(df_mp.index))
    unmap_list.append(len(df_um))
    
df = pd.DataFrame({'sample' : sample_list,
                   'mutations' : pubmut_list,           
                   'mutations with unmappable genes' : unmap_list})
df            

#### Comparison to MHC binding and percentile ranking

In [ ]:
# Get best prediction score for each mutation

pred_m = [
    'NetMHC Score', 
    'NetMHCpan Score', 
    'NetMHCcons Score', 
    'MHCflurry Score', 
    'MHCnuggetsI Score',
    'PickPocket Score']
rank_m = [
    'NetMHC Percentile',
    'NetMHCpan Percentile',
    'NetMHCcons Percentile', 
    'MHCflurry Percentile', 
#    'MHCnuggetsI Percentile',
    'PickPocket Percentile']

for sample in samples:   
    ts_pred = dict()
    for m in pred_m: ts_pred[m] = list()
    ts_rank = dict()
    for m in rank_m: ts_rank[m] = list()
    df_pred = load_predictions(sample)
    df_pred['Mutation'] = df_pred['Mutation'].apply(
        lambda x: '_'.join(
            [str(genemap[x.split('_')[0].upper()] if x.split('_')[0].upper() in genemap.keys() else ''), 
             x.split('_')[1]]
        ))
    df_pred['Mutation'] = df_pred['Mutation'].apply(
        lambda x: '_'.join([x.split('_')[0], ''.join([i for i in x.split('_')[1] if not i.isdigit()])]))
    df_mut = get_mutations_sim(sample)
    for mut in df_mut['Mutation_ID'].unique():
        if mut in df_pred['Mutation'].unique():           
            for m in pred_m:
                minpred = df_pred[df_pred['Mutation'] == mut][m].idxmin()
                if df_pred.loc[minpred][m] <= 500:
                    ts_pred[m].append(df_pred.loc[minpred])
            for m in rank_m:
                minrank = df_pred[df_pred['Mutation'] == mut][m].idxmin()
                if df_pred.loc[minrank][m] <= 2:
                    ts_rank[m].append(df_pred.loc[minrank])
                
    # Compare recall of ranking based on Score (<=500 and top 10) and Percentile (<=2.0 and top 10)
    print('sample', 'tp', 'fn', 'recall', 'method', 'reaction')
    def print_recall(method, reaction, df_list):
        df_list_sort = sorted(df_list, key=lambda x: x[method])[:10]        
        # Store vaccine compositions
        pd.DataFrame([x['Mutation'] for x in df_list_sort]).to_csv(
            os.path.join(basepath, basedir, sample, "{}_{}.txt".format(sample, method.replace(' ', ''))))
        # Get recall
        gt_df = df_tcell[(df_tcell['sample'] == sample) & (df_tcell['type'] == reaction)]
        if len(gt_df) == 0:
            tp = np.NaN
            fn = np.NaN
            recall = np.NaN
        else:
            tp = 0
            fn = 0
            for mut in gt_df['Mutation ID'].tolist():
                if mut in [x['Mutation'] for x in df_list_sort]:
                    tp += 1
                else:
                    fn += 1
            if tp == 0:
                recall = np.NaN
            else:
                recall = tp / (tp + fn)
        print(sample, tp, fn, recall, method, reaction)
    for method in pred_m:
        print_recall(method, 'CD8', ts_pred[method])
    for method in rank_m:
        print_recall(method, 'CD8', ts_rank[method])

#### CD8 Scores -  1000-cells.10x

In [ ]:
get_performance_scores("1000-cells.10x")

#### CD8 Scores -  10000-cells.10x

In [ ]:
get_performance_scores("10000-cells.10x")